In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import json
import os
import random
from pathlib import Path

import dspy
import numpy as np
import pandas as pd
import weave
from datasets import load_dataset
from dspy.evaluate import Evaluate

In [3]:
def set_seed(seed):
    np.random.seed(seed % (2**32 - 1))
    random.seed(seed)


def configure_lm(model, temperature):
    lm = dspy.LM(
        "openai/" + model,
        temperature=temperature,
        cache=False,
        api_base=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
    )
    dspy.configure(lm=lm)


In [4]:
# weave.init(project_name="llm-adaptation")

In [5]:
set_seed(89)

configure_lm('llama-3-8b', 0.0)

In [6]:
from typing import Callable


def compute_generalized_scores(
    pred_triples: list[str], 
    reference_triples: list[str], 
    match_function: Callable[[str, str], bool]
):
    """Compute precision, recall, and F1-score using a customizable match function."""

    pred_set = set(pred_triples)
    reference_set = set(reference_triples)

    if not pred_set and not reference_set:
        return {"precision": 1.0, "recall": 1.0, "f1": 1.0}

    if not pred_set or not reference_set:
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0}

    # Count true positives using the provided match function
    true_positives = sum(any(match_function(pred, ref) for ref in reference_set) for pred in pred_set)
    
    precision = true_positives / len(pred_set)
    recall = true_positives / len(reference_set)
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [7]:
from fuzzywuzzy import fuzz


def fuzzy_match(pred: str, ref: str) -> bool:
    return fuzz.ratio(pred, ref) > 80

def compute_scores(pred_triples: list[str], reference_triples: list[str]):
    exact_scores = {f"exact.{k}": v for k, v in  compute_generalized_scores(pred_triples, reference_triples, lambda x, y: x == y).items()}
    fuzzy_scores = {f"fuzzy.{k}": v for k, v in  compute_generalized_scores(pred_triples, reference_triples, fuzzy_match).items()}
    return {**exact_scores, **fuzzy_scores}

def parse_triples(triples_str: str):
    return [triple.strip() for triple in triples_str.split('\n') if triple.strip()]


@weave.op()
def evaluate_triples(example, pred, trace=None):
    return compute_scores(parse_triples(pred.triples_str), example.triples)['fuzzy.f1']

/home/baris/repos/pipeline-llm-adaptation/.venv/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
from copy import deepcopy


def dynamic_import(module, name):
    import importlib

    return getattr(importlib.import_module(module), name)

def make_optimizer(optimizer_config: dict):
    cls = dynamic_import("dspy.teleprompt", optimizer_config["class"])
    kwargs = deepcopy(optimizer_config["params"])
    if optimizer_config["with_metric"]:
        kwargs["metric"] = evaluate_triples
    return cls(**kwargs)

In [9]:
class EntityRelationExtraction(dspy.Signature):
    """Extract `subject | predicate | object` triples from text."""

    text: str = dspy.InputField()
    triples_str: str = dspy.OutputField(
        desc='The triples extracted from the text. Each triple should be in the format "subject | predicate | object". Triples should be separated by newlines.'
    )


def make_program():
    return dspy.Predict(EntityRelationExtraction)


In [10]:
text = """
Ankara is the capital of Turkey.
Claude Shannon is the father of information theory.
""".strip()

response = make_program()(text=text)
print(text)
print(response)


Ankara is the capital of Turkey.
Claude Shannon is the father of information theory.
Prediction(
    triples_str='Ankara | is the capital of | Turkey\nClaude Shannon | is the father of | information theory'
)


In [11]:
def train(
    dataset_path: str,
    dataset_name: str,
    dataset_split: str,
    model: str,
    temperature: float,
    optimizer_config: dict,
    out: str,
):
    # Set up LM
    configure_lm(model, temperature)

    # Load dataset
    ds = load_dataset(dataset_path, dataset_name, split=dataset_split)
    examples = [dspy.Example(text=x["text"], triples=x["triples"]).with_inputs("text") for x in ds]

    # Create program
    program = make_program()

    # Create and run optimizer
    optimizer = make_optimizer(optimizer_config)
    compile_params = optimizer_config.get("compile_params", {})
    trained_program = optimizer.compile(program, trainset=examples, **compile_params)

    # Save trained program
    Path(out).parent.mkdir(parents=True, exist_ok=True)
    trained_program.save(out)
    return trained_program

In [12]:
def evaluate(
    dataset_path: str,
    dataset_name: str,
    dataset_split: str,
    model: str,
    temperature: float,
    out_dir: str,
    load_from: str | None = None,
):
    # Set up LM
    configure_lm(model, temperature)

    # Load dataset
    ds = load_dataset(dataset_path, dataset_name, split=dataset_split)
    examples = [dspy.Example(text=x["text"], triples=x["triples"]).with_inputs("text") for x in ds]

    # Load program
    program = make_program()
    if load_from:
        program.load(load_from)

    # Evaluate
    evaluator = Evaluate(
        metric=evaluate_triples,
        devset=examples,
        num_threads=16,
        display_progress=True,
        return_outputs=True,
    )
    _, results = evaluator(program)

    # Save results
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    # Process and save detailed results
    processed_results = []
    for example, pred, score in results:
        result = {
            "text": example.text,
            "triples": example.triples,
            "predicted_triples": parse_triples(pred.triples_str),
            **compute_scores(parse_triples(pred.triples_str), example.triples),
        }
        processed_results.append(result)
    
    result_df = pd.DataFrame(processed_results)
    result_df.to_json(out_path / "results.jsonl", orient="records", lines=True)

    # Save aggregate scores
    scores = result_df[["exact.precision", "exact.recall", "exact.f1", "fuzzy.precision", "fuzzy.recall", "fuzzy.f1"]].mean().to_dict()
    with open(out_path / "scores.json", "w") as f:
        json.dump(scores, f, indent=2)

    return results, scores

In [13]:
dataset_path = "bdsaglam/web_nlg-erx-concat"
dataset_name = "release_v3.0_en"

Before prompt optimization

In [14]:
%%capture
results, scores = evaluate(
    dataset_path=dataset_path,
    dataset_name=dataset_name,
    dataset_split="dev[:100]",
    model="llama-3-8b",
    temperature=0.0,
    out_dir="../tmp/erx/dspy/before",
)

In [15]:
scores

{'exact.precision': 0.011607142857142856,
 'exact.recall': 0.012751696832579184,
 'exact.f1': 0.012068311648983919,
 'fuzzy.precision': 0.34766847244117066,
 'fuzzy.recall': 0.3378176513648623,
 'fuzzy.f1': 0.33974898258125846}

Let's optimize the program with DSPy.

In [20]:
optimizer_config = {
    "class": "BootstrapFewShotWithRandomSearch",
    "params": {
        "max_bootstrapped_demos": 8,
        "max_labeled_demos": 8,
        "max_rounds": 16,
        "num_candidate_programs": 16,
        "num_threads": 16,
        "max_errors": 10,
        "metric_threshold": None,
        "stop_at_score": None,
    },
    "with_metric": True,
}

train(
    dataset_path=dataset_path,
    dataset_name=dataset_name,
    dataset_split="train[:100]",
    model="llama-3-8b",
    temperature=0.0,
    optimizer_config=optimizer_config,
    out="../tmp/erx/dspy/trained-program.json",
)

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 16 candidate sets.
Average Metric: 31.60 / 100 (31.6%): 100%|██████████| 100/100 [00:17<00:00,  5.70it/s]

2025/02/14 13:25:14 INFO dspy.evaluate.evaluate: Average Metric: 31.600189246922703 / 100 (31.6%)



New best score: 31.6 for seed -3
Scores so far: [31.6]
Best score so far: 31.6
Average Metric: 32.02 / 100 (32.0%): 100%|██████████| 100/100 [00:17<00:00,  5.81it/s]

2025/02/14 13:25:31 INFO dspy.evaluate.evaluate: Average Metric: 32.01976466366833 / 100 (32.0%)



New best score: 32.02 for seed -2
Scores so far: [31.6, 32.02]
Best score so far: 32.02


...

  4%|▍         | 4/100 [00:08<03:21,  2.10s/it]


Bootstrapped 4 full traces after 4 examples for up to 16 rounds, amounting to 6 attempts.
Average Metric: 41.68 / 100 (41.7%): 100%|██████████| 100/100 [00:19<00:00,  5.06it/s]

2025/02/14 13:34:31 INFO dspy.evaluate.evaluate: Average Metric: 41.68390665799123 / 100 (41.7%)



Scores so far: [31.6, 32.02, 35.02, 0.0, 32.74, 38.9, 42.75, 31.46, 33.81, 35.01, 34.3, 36.15, 31.24, 32.13, 35.68, 36.2, 42.92, 36.6, 41.68]
Best score so far: 42.92
19 candidate programs found.


Predict(EntityRelationExtraction(text -> triples_str
    instructions='Extract `subject | predicate | object` triples from text.'
    text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Text:', 'desc': '${text}'})
    triples_str = Field(annotation=str required=True json_schema_extra={'desc': 'The triples extracted from the text. Each triple should be in the format "subject | predicate | object". Triples should be separated by newlines.', '__dspy_field_type': 'output', 'prefix': 'Triples Str:'})
))

In [18]:
results, scores = evaluate(
    dataset_path=dataset_path,
    dataset_name=dataset_name,
    dataset_split="dev[:100]",
    model="llama-3-8b",
    temperature=0.0,
    out_dir="../tmp/erx/dspy/after",
    load_from="../tmp/erx/dspy/trained-program.json",
)

Average Metric: 39.40 / 100 (39.4%): 100%|██████████| 100/100 [00:18<00:00,  5.47it/s]

2025/02/14 13:10:40 INFO dspy.evaluate.evaluate: Average Metric: 39.40046495393733 / 100 (39.4%)
